<a href="https://colab.research.google.com/gist/mohzeki222/7faebc6a45a511fa33c769ed88ca740b/chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

オリジナルは東北大学大関先生のQA4Uから。
量子アニーリングマシンを使った、組合せ最適化問題の解決を試みるために必要なTipsが満載。
https://altema.is.tohoku.ac.jp/QA4U/

こちらは、プログラム経験がある方用に、一部変更して説明しています。

### **実感の湧く問題を解いてみよう**

少し具体的に意味のある問題を解いてみることにしましょう。


**いくつかの荷物があり、それぞれには重さが異なるものとします。
それを運ぶ2人がいて、重さが均等になるようにその荷物を2つのグループに分けたい。**

どのようにしたら良いでしょうか？
ここで考えなければならないのは、**量子アニーリングマシンへ入力するQUBO行列を作る**ことです。

そこで重要となるのが**数式によるモデリング**です。
ここが量子アニーリングの研究開発を行う上での成長ポイントです。
量子そのものの前に、数理モデリングに挑戦する必要があるのです。

QUBO行列とは、下記のQUBO（二次制約なしバイナリ最適化）問題を定式化した、目的関数$E({\bf x})$の$Q_{ij}$の部分なので、解決する課題をうまくモデリングして、この目的関数の形式に定式化できれば、$Q_{ij}$の部分を取り出すことができる。

\begin{equation}
E({\bf x}) = \sum_{i=1}^N Q_{ij} x_i x_j
\end{equation}



重さを持ついくつかの荷物があるというのだから、その重さを$w_i$としましょう。
$N$個あるとして、合計した重さは$W=\sum_{i=1}^N w_i$です。式は、すべての荷物の重さの合計$W$は、各荷物の重さを格納したリスト$w_i$の1番目からＮ番目を合計するという意味です。


（こうやって何も与えられていないところで**自分で文字式を立てる**ところから訓練です）


2人のうちAさんがその荷物を取る場合$x_i=1$として、取らない場合は$x_i=0$とすると、
Aさんが持つ荷物の重さの合計は、
\begin{equation}
W_A = \sum_{i=1}^N w_i x_i
\end{equation}

Σ記号に慣れていない方のために、少し解説します。

この式は、Aさんが持つ荷物の重さの合計$W_A$は、各荷物の重さを格納するリスト$w_i$とAさんがその荷物を持つかどうかを1と0で表現したリスト$x_i$の各要素を乗算した値を合計するという意味です。

例えば、荷物が８つあって、各荷物の重さと、Aさんが持つ荷物のリストがそれぞれ、下記のの場合

\begin{equation}
w_i = [5,6,4,7,6,2,8,4]
\end{equation}
\begin{equation}
x_i = [0,1,1,0,1,1,0,0]
\end{equation}

$w_i$と$x_i$の要素をそれぞれ乗算した値$w_i$$x_i$は、下記となる。

\begin{equation}
w_i x_i = [0,6,4,0,6,2,0,0]
\end{equation}

$W_A$は、リスト$w_i$$x_i$の要素の合計なので、Aさんの持つ荷物の合計の重さは18となる。





逆にBさんは残りの荷物を持つので
\begin{equation}
W_B = W - W_A
\end{equation}
となります。
これらが等しくなるというのだから、
\begin{equation}
W_A - W_B
\end{equation}
という引き算をしたズレが$0$になれば完璧です。
もしくは非常に小さいものとなってくれれば嬉しい。
ただ$W_A$がわずかに大きくても仕方ないし、小さくても仕方ない。正負の値どちらでも良いからとにかく**ズレの大きさ**ができるだけ小さいことが望ましいというわけです。
できるだけ小さい、すなわちズレの大きさが最小になるような組み合わせを見つければ良いですよね。
そうするとコスト関数として、次のようなものを考えてみましょう。
\begin{equation}
E({\bf x}) = \left( W_A - W_B \right)^2 = \left( 2W_A - W \right)^2
\end{equation}

ソフトウェアエンジニアなら**「絶対値でいいじゃん！」**って考える方も多いと思いますが、量子アニーリングマシンで解く最適化問題が、QUBO（二次制約なしバイナリ最適化）問題であることを思い出してみよう。定式化した目的関数は、二次関数である必要があるので、ここは**二乗にしておく方が都合が良い**のです。

ここに$W_A$の具体的な形として先ほど準備しておいた形を入れてみましょう。
\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)^2
\end{equation}

何か近い形になってきましたね。二乗をするというのは同じものを掛け算するという意味です。
シグマ記号は嫌らしいけれども意味はとにかく足し算をするというものでした。
下にある$i=1$は$i$という文字を$1$から動かして上にある$N$まで変えて足し算してくださいね、ということです。
だったら$i$という文字は仮置きをしているだけですから、別の文字を使っても良いですね。


\begin{equation}
E({\bf x}) = \left( 2\sum_{i=1}^N w_i x_i - W \right)\left( 2\sum_{j=1}^N w_j x_j - W \right)
\end{equation}

この掛け算を展開してみましょう。

\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 2W\sum_{i=1}^N w_i x_i  - 2W\sum_{j=1}^N w_j x_j + W^2
\end{equation}

ここで第二項と第三項で同じ和が2つ出ていますので、まとめておきましょう。
\begin{equation}
E({\bf x}) = 4\sum_{i=1}^N\sum_{j=1}^N w_iw_j x_ix_j - 4W\sum_{i=1}^N w_i x_i + W^2
\end{equation}


$x_ix_j$という部分が出てきましたね。その係数は$4w_iw_j$で、この部分がQUBO行列に相当します。$x_i$が$0$と$1$のどちらかの値を持つことから、$x_i$が$x_ix_i$と同じことを思いつくと、第二項は、$-4W\sum_{i=1}^N w_i x_ix_i$という意味で、QUBO行列のうち添え字が同じ$i$と$i$のとき、対角成分のことを示していることがわかります。
これらの事実からQUBO行列を作るプログラムの発想ができます。

In [85]:
import numpy as np

In [86]:
N = 10
w = np.random.rand(N)

まず例えば$N=10$個の荷物について、その重さを適当な乱数で決めることにしましょう。

np.random.rand()で$0$から$1$の範囲にある適当な数値が出ます。

この係数からQUBO行列を作ります。
まず最初に全重量の計算です。

In [87]:
W = np.sum(w)

numpyのsum()を利用すれば全重量の合計が計算できます。

In [88]:
Q = np.zeros(N**2).reshape(N,N)

まずはQUBO行列を格納する場所を用意しましょう。np.zeros()はその名の通り、全成分を0で埋めたものを並べて作ります。これを.reshape(N,N)でN×Nの行列に整形します。

In [89]:
for i in range(N):
  for j in range(N):
    Q[i][j] = 4*w[i]*w[j]

まず第一項の計算をここで実行しています。for文を使って、iとjを動かしながら、$Q_{ij}$すなわちQ[i][j]に値を入れていきます。その値は$4w_iw_j$ですから、その結果を入れていきます。
次に第二項をQUBO行列の対角成分に追加しましょう。

In [90]:
for i in range(N):
  Q[i][i] = Q[i][i] - 4*W*w[i]

ここで注意して欲しいのが、第一項で計算した結果に追加するので、元からあるQ[i][i]に引き算をするようにしています。

これでQUBO行列の作成完了です。

### **QUBO行列の設定方法について**

QUBO行列はだんだんと巨大なものになってくると、そのデータ転送にも時間がかかるようになり、せっかくの量子アニーリングマシンのパワーを台無しにしてしまうことがあります。
データの転送量を抑えるためにも、不要な成分についてはその情報を送らないようにしておくと良いでしょう。
python上ではnumpyによるarray形式で行列を扱うことが多いのですが、代わりにdict形式でデータを送りましょう。


In [91]:
Qdict = {}
for i in range(N):
  for j in range(N):
    if Q[i][j] != 0.0:
      Qdict[(i,j)] = Q[i][j]

dict形式の初期化は{}で中身は空欄にしておくだけです。
Qdict[(i,j)]のように(i,j)でどの成分に値があるのかを指定して、その値を代入します。
ここではif文を使い、ゼロではないところだけ、Qdictのデータとして格納しています。
出来上がったものを確認したい場合にはQdictとそのまま打って実行したり、print(Qdict)と実行してみましょう。

In [92]:
print(Qdict)

{(0, 0): -9.6943091646132, (0, 1): 1.1820357015673357, (0, 2): 2.543390878188608, (0, 3): 2.708469299334047, (0, 4): 0.9488193394128392, (0, 5): 0.3561772956519539, (0, 6): 0.3903797170448637, (0, 7): 1.0815379789913604, (0, 8): 0.26074105613126974, (0, 9): 0.22275789829092374, (1, 0): 1.1820357015673357, (1, 1): -6.1748991715687165, (1, 2): 1.4918227734073224, (1, 3): 1.5886493171268945, (1, 4): 0.5565288098357328, (1, 5): 0.20891535217058152, (1, 6): 0.22897674012993274, (1, 7): 0.6343747637065298, (1, 8): 0.1529373439350913, (1, 9): 0.13065836968929667, (2, 0): 2.543390878188608, (2, 1): 1.4918227734073224, (2, 2): -11.568416718201584, (2, 3): 3.41830299750124, (2, 4): 1.1974852337442128, (2, 5): 0.449523986728711, (2, 6): 0.4926901542750557, (2, 7): 1.3649866795265568, (2, 8): 0.32907588576481467, (2, 9): 0.28113812906506486, (3, 0): 2.708469299334047, (3, 1): 1.5886493171268945, (3, 2): 3.41830299750124, (3, 3): -12.097398683461751, (3, 4): 1.2752078415536177, (3, 5): 0.4787002767

どこの成分に重要な非零の行列成分があるのかを指定する形になっています。
上記の問題では非零成分のない問題になっていますので影響はさほどありませんが、
基本的なテクニックとして知っておくと良いでしょう。

（正直この入力の違いだけでハイブリッドソルバーなどでは如実に性能が変わります）

### **シミュレータを活用しよう**

さて上記のように用意した量子アニーリングマシンは、利用回数には制限があり、大事に使いたいところでしょう。演習の際には豊富なマシンタイムを利用することのできるAPI tokenを発行する予定ですが、講義の間の試し利用の場合には、代わりになるシミュレータを利用すると良いでしょう。
その一つが**株式会社Jijの開発するOpenJij**です。

OpenJijは量子アニーリングマシンのシミュレータを搭載するオープンソースソフトウェアです。


基本的な利用方法は、これまでと同じようにQUBO行列を作ったのちにsamplerに投入するだけです。その際にOpenJijのsamplerを利用します。

その前にまずはOpenJijのインストールが必要です。

In [93]:
pip install openjij

再びpip installを利用して、OpenJijをインストールします。
その後にimport SQASamplerを実行してsamplerの準備を行いましょう。

In [94]:
from openjij import SQASampler
sampler = SQASampler()

準備はこれだけです。SQAというのはシミュレーテッド量子アニーリングというもので、
**量子モンテカルロ法**という計算技術を活用して、**量子アニーリングのシミュレーション**を行っています。
オプションでそのシミュレーションのパラメータ等を設定することができますが、とりあえず前に進めていきましょう。

In [95]:
sampleset = sampler.sample_qubo(Qdict, num_reads=10)

In [96]:
print(sampleset.record)

[([0, 0, 1, 1, 0, 1, 0, 0, 0, 1], -17.00930221, 1)
 ([1, 1, 1, 0, 0, 1, 0, 0, 0, 1], -16.90237259, 1)
 ([1, 1, 0, 1, 0, 0, 1, 0, 0, 0], -16.91292932, 1)
 ([0, 0, 0, 1, 1, 1, 1, 1, 1, 1], -17.00312635, 1)
 ([0, 1, 0, 1, 1, 1, 0, 0, 1, 1], -16.99425621, 1)
 ([0, 1, 0, 1, 0, 0, 0, 1, 1, 0], -16.81761453, 1)
 ([1, 1, 0, 0, 0, 1, 0, 1, 1, 1], -16.74057911, 1)
 ([0, 1, 1, 0, 0, 1, 0, 1, 0, 1], -16.90049724, 1)
 ([0, 1, 0, 1, 1, 1, 1, 0, 0, 0], -16.97370929, 1)
 ([1, 0, 0, 1, 1, 0, 0, 0, 1, 0], -17.00653013, 1)]


OpenJijではdict形式で問題を受け付けますので、注意してください。

（先程の手順でnumpy array形式だったものをdict形式に直しておきましょう）

結構いい答えが出てきたのではないでしょうか。これでとりあえずは十分ですよね。
ただ問題のサイズが大きくなるにつれて結果が次第に悪化してくることがありますのでご注意を。
それはパラメータの設定で改善する可能性があります。